# Prognosemodelle

Nachdem Du im vorherigen Schritt die explorative Phase des Datensets abgeschlossen hast, sollst du nun für einige der Produkte ein Prognosemodell entwickeln. Schneide dazu bitte zuerst die letzten 30 Tage der Daten ab und behalte sie als Testdaten (Auf Basis dieser Daten bewertest du dein endgültiges Modell)

Gehe bei der Entwicklung iterativ vor. Das bedeutet:
- Steigende Modellkomplexitäten
- Übertragen der Erkenntnisse aus der explorativen Phase mit in die Modelle
- Ggf hinzuziehen von wichtigen Daten wie "Holidays"
- Track deine Kennzahlen in geeignetem Maße (Nicht löschen reicht schon aus)

Hast du Daten im vorherigen Schritt bereinigt, so baue auf diesen Bereinigungen auf.
Welche Modelle du für diese Aufgabe verwendest, steht dir frei. Ein paar Tipps:
- das `Darts` Package bietet eine vielzahl von Algorithmen. Dabei musst du dein Datenset nicht jedes mal in eine neue Form bringen und kannst auf altbewährte Methoden zurückgreifen.
- Entscheidest du dich für `Prophet`, so bietet sich ein Blick in die offizielle Dokumentation an. Dort wird exakt erläutert, wie externe Einflüsse wie Holidays modelliert werden können. 
**Achtung**: Die Interfaces von `darts.models.Prophet` und `prophet.Prophet` unterscheiden sich.

In [1]:
# Your Code here
import os
import warnings
from datetime import datetime

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import Latex, Markdown, display

warnings.filterwarnings("ignore")

data_path = "../data/tsa02/store-sales"

train_df = pd.read_csv(os.path.join(data_path, "train.csv"))
test_df = pd.read_csv(os.path.join(data_path, "test.csv"))
holiday_df = pd.read_csv(os.path.join(data_path, "holidays_events.csv"))
oil_df = pd.read_csv(os.path.join(data_path, "oil.csv"))
stores_df = pd.read_csv(os.path.join(data_path, "stores.csv"))

In [2]:
def load_data(data_path):
    train_df = pd.read_csv(os.path.join(data_path, "train.csv"))
    train_df["date"] = pd.to_datetime(train_df["date"])
    test_df = pd.read_csv(os.path.join(data_path, "test.csv"))
    test_df["date"] = pd.to_datetime(test_df["date"])
    holiday_df = pd.read_csv(os.path.join(data_path, "holidays_events.csv"))
    holiday_df["date"] = pd.to_datetime(holiday_df["date"])
    oil_df = pd.read_csv(os.path.join(data_path, "oil.csv"))
    oil_df["date"] = pd.to_datetime(oil_df["date"])
    stores_df = pd.read_csv(os.path.join(data_path, "stores.csv"))
    return train_df, test_df, holiday_df, oil_df, stores_df


def clean_data(df, z_threshold=3):
    df_clean = df.copy()
    # remove duplicates
    df_clean = df_clean.drop_duplicates()
    # calculate z-score on (store_nbr + family) - date level
    df_clean["z_score"] = df_clean.groupby(["store_nbr", "family"])["sales"].transform(
        lambda x: (x - x.mean()) / x.std()
    )
    df_clean["z_score"] = df_clean["z_score"].fillna(0)

    # fill with median
    df_clean["sales"] = np.where(
        df_clean["z_score"].abs() > z_threshold,
        df_clean.groupby(["store_nbr", "family"])["sales"].transform("median"),
        df_clean["sales"],
    )

    # remove z_score column
    df_clean = df_clean.drop(columns=["z_score"])

    # reset index
    df_clean = df_clean.reset_index(drop=True)

    print(f"Before cleaning: {df.shape}")
    print(f"After cleaning: {df_clean.shape}")

    return df_clean


def get_top_families(df, n=5):
    family_sales_df = df.groupby("family")["sales"].sum().reset_index()
    family_sales_df = family_sales_df.sort_values(by="sales", ascending=False)
    top_families = family_sales_df["family"].head(n).tolist()
    return top_families


def get_top_stores(df, n=5):
    store_sales_df = df.groupby("store_nbr")["sales"].sum().reset_index()
    store_sales_df = store_sales_df.sort_values(by="sales", ascending=False)
    top_stores = store_sales_df["store_nbr"].head(n).tolist()
    return top_stores


def get_store_sales(df):
    store_sales_df = df.groupby(["store_nbr", "date"])["sales"].sum().reset_index()
    store_sales_df = store_sales_df.sort_values(by="date")
    return store_sales_df


def get_family_sales(df):
    family_sales_df = df.groupby(["family", "date"])["sales"].sum().reset_index()
    family_sales_df = family_sales_df.sort_values(by="date")
    return family_sales_df


def plot_sales_per_family(df):
    family_sales_df = df.groupby("family")["sales"].sum().reset_index()
    family_sales_df = family_sales_df.sort_values(by="sales", ascending=False)
    fig = px.bar(
        family_sales_df, x="family", y="sales", title="Sales per Family", log_y=True
    )
    return fig


def plot_sales_per_store(df):
    store_sales_df = df.groupby("store_nbr")["sales"].sum().reset_index()
    store_sales_df = store_sales_df.sort_values(by="sales", ascending=False)
    store_sales_df["store_nbr"] = store_sales_df["store_nbr"].astype(str)
    fig = px.bar(store_sales_df, x="store_nbr", y="sales", title="Sales per Store")
    return fig


def plot_sales_per_family_over_time(df, top_families):
    top_families_df = df[df["family"].isin(top_families)]
    top_families_sales_df = (
        top_families_df.groupby(["family", "date"])["sales"].sum().reset_index()
    )
    fig = px.line(
        top_families_sales_df,
        x="date",
        y="sales",
        color="family",
        title="Top Families Sales",
    )
    return fig


def plot_sales_per_family_over_time_with_holidays(df, top_families, holiday_df):
    top_families_df = df[df["family"].isin(top_families)]
    top_families_sales_df = (
        top_families_df.groupby(["family", "date"])["sales"].sum().reset_index()
    )
    top_families_sales_df = top_families_sales_df.merge(
        holiday_df, on="date", how="left"
    )
    top_families_sales_df["is_holiday"] = top_families_sales_df["type"] == "Holiday"
    fig = px.line(
        top_families_sales_df,
        x="date",
        y="sales",
        color="family",
        title="Top Families Sales",
    )
    fig.add_scatter(
        x=top_families_sales_df[top_families_sales_df["is_holiday"]]["date"],
        y=top_families_sales_df[top_families_sales_df["is_holiday"]]["sales"],
        mode="markers",
        marker=dict(color="red", size=8),
        name="Holiday",
    )
    return fig


def plot_sales_per_store_over_time(df, store_nbr):
    store_family_df = (
        df[(df["store_nbr"] == store_nbr)].groupby("date")["sales"].sum().reset_index()
    )
    store_family_df = store_family_df.sort_values(by="date")
    store_family_df["date"] = pd.to_datetime(store_family_df["date"])
    fig = px.line(
        store_family_df, x="date", y="sales", title=f"Store {store_nbr} Sales"
    )
    return fig


def plot_family_sales_for_store(df, store_nbr):
    store_family_df = df[(df["store_nbr"] == store_nbr)]
    fig = px.line(
        store_family_df,
        x="date",
        y="sales",
        color="family",
        title=f"Store {store_nbr} Family Sales",
    )
    return fig


def slice_df_by_date(df, start_date, end_date):
    return df[(df["date"] >= start_date) & (df["date"] <= end_date)]

def decompose_time_series(df, period):
    from statsmodels.tsa.seasonal import seasonal_decompose

    result = seasonal_decompose(df["sales"], model="additive", period=period)
    return result

def check_stationarity(df):
    # check with adfuller, histogram
    from statsmodels.tsa.stattools import adfuller

    result = adfuller(df["sales"])
    mean = df["sales"].mean()
    std = df["sales"].std()
    fig = px.histogram(df, x="sales", title="Sales Histogram")
    return result, fig


def print_adfuller_result(result):
    print(f"ADF Statistic: {result[0]}")
    print(f"p-value: {result[1]}")
    print("Critical Values:")
    for key, value in result[4].items():
        print(f"\t{key}: {value}")

In [3]:
train_df, test_df, holiday_df, oil_df, stores_df = load_data(data_path)
cl_train_df = clean_data(train_df)
top_3_families = get_top_families(cl_train_df, n=3)
top_3_stores = get_top_stores(cl_train_df, n=3)
store_sales_df = get_store_sales(cl_train_df)
family_sales_df = get_family_sales(cl_train_df)

display(Markdown("### Top 3 Families"))
print(top_3_families)
display(Markdown("### Top 3 Stores"))
print(top_3_stores)

display(Markdown("### Plots for Sales per Family and Store (after cleaning)"))
fig = plot_sales_per_family(cl_train_df)
fig.show()

fig = plot_sales_per_store(cl_train_df)
fig.show()

display(Markdown("### Plots for Store Sales over Time"))
fig = plot_sales_per_store_over_time(cl_train_df, 44)
fig.show()

Before cleaning: (3000888, 6)
After cleaning: (3000888, 6)


### Top 3 Families

['GROCERY I', 'BEVERAGES', 'PRODUCE']


### Top 3 Stores

[44, 45, 47]


### Plots for Sales per Family and Store (after cleaning)

### Plots for Store Sales over Time

In [115]:
# df_total = (
#     cl_train_df[(cl_train_df["store_nbr"] == 44)]
#     .groupby("date")["sales"]
#     .sum()
#     .reset_index()
# )

# get a top product family of store 44
df_total = cl_train_df[(cl_train_df["store_nbr"] == 3)]
top_family = df_total.groupby("family")["sales"].sum().idxmax()
top_family = "FROZEN FOODS"

print(f"Top family for store 44: {top_family}")

df_total = df_total[df_total["family"] == top_family]
df_total = df_total.groupby("date")["sales"].sum().reset_index()

Top family for store 44: FROZEN FOODS


In [117]:
total_result, total_fig = check_stationarity(df_total)
print("Total Sales:")
print_adfuller_result(total_result)
total_fig.show()

Total Sales:
ADF Statistic: -5.820089029727789
p-value: 4.2027339577549183e-07
Critical Values:
	1%: -3.434300212992577
	5%: -2.863284793874921
	10%: -2.567698886736967


In [122]:
# cut last 30 days for testing
days_for_test = 45
df_train = df_total[:-days_for_test]
df_test = df_total[-days_for_test:]

In [123]:
# plot train and test and color test red
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_train["date"], y=df_train["sales"], mode="lines", name="Train"))
fig.add_trace(go.Scatter(x=df_test["date"], y=df_test["sales"], mode="lines", name="Test", line=dict(color="red")))
fig.show()

In [124]:
import darts.models as dm
from darts.metrics import mape, mse


def train(model, train_series, test_series, params={}):
    # model is str
    if isinstance(model, str):
        model = getattr(dm, model)(**params)
    model.fit(train_series)
    prediction = model.predict(len(test_series))
    _mape = mape(test_series, prediction)
    _mse = mse(test_series, prediction)
    print(f"Model: {model}")
    print(f"MAPE: {_mape}")
    print(f"MSE: {_mse}")
    return prediction.values().flatten()

def train_rnn(train_series, test_series, params={}):
    from darts.models import RNNModel
    
    model = RNNModel(**params, input_chunk_length=30, training_length=30)
    model.fit(train_series)
    prediction = model.predict(len(test_series))
    # _mape = mape(test_series, prediction)
    # _mse = mse(test_series, prediction)
    # print(f"Model: {model}")
    # print(f"MAPE: {_mape}")
    # print(f"MSE: {_mse}")
    
    return prediction

def train_transformer(train_series, test_series, params={}):
    from darts.models import TiDEModel
    
    model = TiDEModel(**params, input_chunk_length=30, output_chunk_length=14)
    model.fit(train_series)
    prediction = model.predict(len(test_series))
    # _mape = mape(test_series, prediction)
    # _mse = mse(test_series, prediction)
    # print(f"Model: {model}")
    # print(f"MAPE: {_mape}")
    # print(f"MSE: {_mse}")
    
    return prediction

In [126]:
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler

train_series = TimeSeries.from_dataframe(
    df_train, time_col=None, value_cols="sales", freq="D"
)
test_series = TimeSeries.from_dataframe(
    df_test, time_col=None, value_cols="sales", freq="D"
)

scaler = Scaler()
train_series = scaler.fit_transform(train_series)
test_series = scaler.transform(test_series)

# predictions_arima = train("ARIMA", train_series, test_series)
# predictions_random_forest = train(
#     "RandomForest", train_series, test_series, params={"lags": 7, "n_estimators": 100}
# )
predictions_rnn = train_rnn(
    train_series,
    test_series,
    params={
        "n_epochs": 30,
        "hidden_dim": 20,
        "n_rnn_layers": 2,
        "dropout": 0.1,
        "optimizer_kwargs": {"lr": 3e-3},
    },
)


predictions_transformer = train_transformer(
    train_series,
    test_series,
    params={
        "n_epochs": 30,
        "num_encoder_layers": 3,
        "num_decoder_layers": 3,
        "hidden_size": 32,
        "optimizer_kwargs": {"lr": 1e-4},
    },
)


fig = go.Figure()
train_y = train_series.pd_series()
fig.add_trace(
    go.Line(
        x=df_train["date"][-60:], y=df_train["sales"][-60:], mode="lines", name="Train"
    )
)
fig.add_trace(
    go.Line(
        x=df_test["date"],
        y=df_test["sales"],
        mode="lines",
        name="Test",
        line=dict(color="red"),
    )
)
# fig.add_trace(
#     go.Line(
#         x=df_test["date"],
#         y=predictions_arima,
#         mode="lines",
#         name="ARIMA",
#         line=dict(color="green"),
#     )
# )
# fig.add_trace(
#     go.Line(
#         x=df_test["date"],
#         y=predictions_random_forest,
#         mode="lines",
#         name="Random Forest",
#         line=dict(color="blue"),
#     )
# )

fig.add_trace(
    go.Line(
        x=df_test["date"],
        y=scaler.inverse_transform(predictions_rnn).values().flatten(),
        mode="lines",
        name="RNN",
        line=dict(color="purple"),
    )
)

fig.add_trace(
    go.Line(
        x=df_test["date"],
        y=scaler.inverse_transform(predictions_transformer).values().flatten(),
        mode="lines",
        name="Transformer",
        line=dict(color="orange"),
    )
)


fig.show()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | rnn             | RNN              | 1.3 K  | train
6 | V               | Linear           | 21     | train
-------------------------------------------------------------
1.3 K     Trainable params
0         Non-trainable params
1.3 K     Total params
0.005     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


Epoch 0:   0%|          | 0/51 [00:00<?, ?it/s] 

Epoch 29: 100%|██████████| 51/51 [00:00<00:00, 59.92it/s, train_loss=0.00348]

`Trainer.fit` stopped: `max_epochs=30` reached.


Epoch 29: 100%|██████████| 51/51 [00:00<00:00, 59.92it/s, train_loss=0.00348]


...

`Trainer.fit` stopped: `max_epochs=30` reached.


Epoch 29: 100%|██████████| 50/50 [00:00<00:00, 58.71it/s, train_loss=0.00664]


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 116.31it/s]


c:\Storage\Workspace\Python\Master\summerschool24\.venv\Lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


c:\Storage\Workspace\Python\Master\summerschool24\.venv\Lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


c:\Storage\Workspace\Python\Master\summerschool24\.venv\Lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


